# PilotNet SDNN Example

This tutorial demonstrates how to use __lava__ to perform inference on a PilotNet SDNN on both CPU and Loihi 2 neurocore.

![PilotNet Inference](images/pilotnet_sdnn.PNG)

The network receives video input, recorded from a dashboard camera of a driving car (__Dataloader__). The data is encoded efficiently as the difference between individual frames (__Encoder__). The data passes through the PilotNet SDNN, which was trained with __lava-dl__ and is built using its __Network Exchange__ module (netx.hdf5.Network), which automatically generates a Lava process from the training artifact. The network estimates the angle of the steering wheel of the car, which is decoded from the network's raw output (__Decoder__) and sent to a visualization (__Monitor__) and logging system (__Logger__).

The core of the tutorial is lava-dl's Network Exchange module, which is available as `lava.lib.dl.netx.{hdf5, blocks, utils}`.
* `hdf5` implements automatic network generation.
* `blocks` implements individual layer blocks.
* `utils` implements hdf5 reading utilities. 

In addition, it also demonstrates how different lava processes can be connected with each other for real time interaction between them even though the underlying processes can be run on various backends, including Loihi 2.

Switching between Loihi 2 hardware and CPU simulation is as simple as changing the run configuration settings.

In [1]:
import numpy as np
import os

from lava.magma.core.run_conditions import RunSteps
from lava.proc.io.encoder import Compression
from lava.proc.io.sink import RingBuffer as SinkBuffer
from lava.proc.io.dataloader import SpikeDataloader
from lava.proc.plot.streaming import Figure, Raster, ImageView, LinePlot

from lava.lib.dl.netx.hdf5 import Network

from dataset import PilotNetDataset
from utils import (
    PilotNetEncoder, PilotNetDecoder, PilotNetMonitor,
    CustomHwRunConfig, CustomSimRunConfig,
    get_input_transform
)

from lava.utils import loihi

# Import modules for Loihi2 execution

Check if Loihi2 compiker is available and import related modules.

In [2]:
if loihi.is_installed():
    loihi.use_slurm_host(partition='oheogulch')
    print(f'Running on {loihi.host}')
else:
    print('Lava-Loihi is not installed. Running on CPU.')

Lava-Loihi is not installed. Running on CPU.


## Create network block

PilotNet SDNN is described by the hdf5 file interface `network.net` exported after training. You can refer to the training tutorial that trains the networks and exports hdf5 file interface at [`tutorials/lava/lib/dl/slayer/pilotnet/train.ipynb`](https://github.com/lava-nc/lava-dl/blob/main/tutorials/lava/lib/dl/slayer/pilotnet/train.ipynb)

A network block can be created by simply instantiating `netx.hdf5.Network` with the path of the desired hdf5 network description file.
* The input layer is accessible as `net.in_layer`.
* The output layer is accessible as `net.out_layer`.
* All the constituent layers are accessible as a list: `net.layers`.

![PilotNet Inference](images/pilotnet_sdnn_network.PNG)

In [3]:
net = Network(net_config='network.net', skip_layers=1)
print(net)

|   Type   |  W  |  H  |  C  | ker | str | pad | dil | grp |delay|
|Conv      |   99|   32|   24| 3, 3| 2, 2| 0, 0| 1, 1|    1|False|
|Conv      |   49|   15|   36| 3, 3| 2, 2| 0, 0| 1, 1|    1|False|
|Conv      |   24|    7|   48| 3, 3| 2, 2| 0, 0| 1, 1|    1|False|
|Conv      |   22|    4|   64| 3, 3| 1, 2| 0, 1| 1, 1|    1|False|
|Conv      |   20|    2|   64| 3, 3| 1, 1| 0, 0| 1, 1|    1|False|
|Dense     |    1|    1|  100|     |     |     |     |     |False|
|Dense     |    1|    1|   50|     |     |     |     |     |False|
|Dense     |    1|    1|   10|     |     |     |     |     |False|
|Dense     |    1|    1|    1|     |     |     |     |     |False|


## Set execution parameters
Configure number of samples, execution timesteps, and readout offset.

In [4]:
num_samples = 20000
steps_per_sample = 1
num_steps = num_samples + len(net.layers)
out_offset = len(net.layers) + 3
sample_offset = 10550
print(f'Run {num_steps} steps starting with sample {sample_offset}')

Run 20009 steps starting with sample 10550


## Create Dataset instance
Typically the user would write it or provide it.

In [5]:
transform = get_input_transform(net.net_config)
print(f'Input Layer Transform: {transform}')
dataset = PilotNetDataset(
    path='../data',
    size=net.inp.shape[:2],
    transform=transform,
    visualize=True,
    sample_offset=sample_offset,
)
print(f'Dataset initialized. Loaded {len(dataset)} samples.')

Input Layer Transform: {'weight': 64, 'bias': 0}
Dataset initialized. Loaded 45406 samples.


## Create Dataloader
The dataloader process reads data from the dataset objects and sends out the input frame and ground truth as spikes.

![PilotNet Inference](images/pilotnet_sdnn_dataloader.PNG)

In [6]:
dataloader = SpikeDataloader(dataset=dataset)

## Create Input Encoder

The input encoder process does frame difference of subsequent frames to sparsify the input to the network.

For Loihi execution, it additionally compresses and sends the input data to the Loihi 2 chip.

![PilotNet Inference](images/pilotnet_sdnn_encoder.PNG)

In [7]:
compression = Compression.DELTA_SPARSE_8 if loihi.host else Compression.DENSE
input_encoder = PilotNetEncoder(shape=net.inp.shape,
                                net_config=net.net_config,
                                compression=compression)

## Create Output Decoder

The output decoder process receives the output from the network and applies proper scaling to decode the steering angle prediction.

For Loihi execution, it additionally communicates the network's output spikes from the Loihi 2 chip.

![PilotNet Inference](images/pilotnet_sdnn_decoder.PNG)

In [8]:
output_decoder = PilotNetDecoder(shape=net.out.shape)

# Connect the Model Processes

TODO: Update graphic
![PilotNet Inference](images/pilotnet_sdnn.PNG)

In [9]:
dataloader.s_out.connect(input_encoder.inp)
input_encoder.out.connect(net.inp)
net.out.connect(output_decoder.inp)

## Create a Streaming Figure to Visualize the Input, Output Spikes, and Output Value

TODO: Update graphic
![PilotNet Inference](images/pilotnet_sdnn_monitors.PNG)

In [10]:
image = ImageView(shape=dataloader.s_out.shape, bias=transform['bias'],
                  range=transform['weight'], transpose=[1, 0, 2], subplot=121)
raster = Raster(shape=net.layers[-2].out.shape, subplot=222)
lines = LinePlot(length=1000, min=-np.pi, max=np.pi, num_lines=2, subplot=224)
figure = Figure(plots=[image, raster, lines])

# Connect the Plot Inputs to the Corresponding Model Ports

In [11]:
dataloader.s_out.connect(image.img_in)
net.layers[-2].out.connect(raster.spk_in)
output_decoder.out.connect(lines.y_in[0])
dataloader.ground_truth.connect(lines.y_in[1])

## Run the network

Switching between Loihi 2 hardware and CPU simulation is as simple as changing the run configuration settings.

![PilotNet Inference](images/pilotnet_sdnn_backends.PNG)

In [12]:
print('Compiling network...', end='')
run_config = CustomHwRunConfig() if loihi.host else CustomSimRunConfig()
net.create_runtime(run_cfg=run_config)
print('Done.')

print('Running network...', end='')
net.run(condition=RunSteps(num_steps=num_steps, blocking=False))
figure.show()
print('Done.')
if net.runtime._is_running:
    net.stop()

KeyboardInterrupt: 